# Lecture Plan

1. Chains
   - Simple Sequenctial Chain
   - Sequential Chain
   - Router Chain
   - Math Chain
   - Tranform Chain
2. Document Loaders
3. Transformers
4. Text Embeddings
5. Vector Store
6. Retreivers
7. Multi Query Retreival


---

### LangChain Expression Language (LCEL)


- Easy way to compose chains together using Runnable Interface
  
Provides:

- Streaming
- Async
- Parallel Execution
- Retires and Fallbacks
- Input and Output Schemas
- Integration with langsmith

#### Runnable Interface

- strem
- invoke
- batch

Async:

- astrem
- ainvoke
- abatch


[LCEL Reference](https://python.langchain.com/docs/expression_language/interface)

## Runnable Interface


1. Runnable Parallel: used to define  and run multiple values and operations in parallel.

2. Runnble Passthrough: used to take the input and pass it through 

3. Runnable Lambda: used to turn the python functions to pipe compatable functions

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### LLM Chain

In [2]:
from langchain.chat_models.openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

c:\TrainingMaterial\generative-ai\genai-material\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms.deepinfra import DeepInfra
from langchain_experimental.chat_models import Llama2Chat
llm = DeepInfra(
    model_id="meta-llama/Llama-2-70b-chat-hf"
)

model = Llama2Chat(llm=llm)

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a code assistant that generates code for users input"),
        HumanMessagePromptTemplate.from_template("provide code for {text} in {language}")
    ]
)

In [11]:
# LLM Chain

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser

chain.invoke({'text':'print prime numbers', 'language':'python'})

'Sure! Here\'s a code snippet that prints all prime numbers within a given range in Python:\n\n```python\ndef is_prime(num):\n    if num < 2:\n        return False\n    for i in range(2, int(num ** 0.5) + 1):\n        if num % i == 0:\n            return False\n    return True\n\nstart = int(input("Enter the starting number: "))\nend = int(input("Enter the ending number: "))\n\nprint("Prime numbers between", start, "and", end, "are:")\nfor num in range(start, end + 1):\n    if is_prime(num):\n        print(num)\n```\n\nIn this code, we define a helper function `is_prime()` that checks whether a number is prime or not. Then, we take user input for the starting and ending numbers of the range. Finally, we iterate through the range and print all the prime numbers within that range.'

### Simple Sequential Chain

![Simple Sequential Chain](./images/SimpleSequentialChain.PNG)

In [41]:
company_name_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are an assistant that provides a single company name based on the description provided by the user"),
        HumanMessagePromptTemplate.from_template("{text}")

    ]
)

slogan_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="genrate a slogan from company name provided by the user and print it along with the name of the company"),
        HumanMessagePromptTemplate.from_template("{company_name}")

    ]
)

In [42]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    company_name = company_name_prompt | model,
    company_slogan = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 
)
# chain = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 


In [43]:
chain.invoke({'text':'a company that sells flying cars'})

{'company_name': AIMessage(content='  Sure! Based on your description, the company name that comes to mind is "Terrafugia". Terrafugia is a company that designs and manufactures flying cars, also known as personal aerial vehicles or PAVs. The company was founded in 2006 and is based in Woburn, Massachusetts, USA. Terrafugia\'s mission is to create practical and safe flying cars that can be used for personal transportation and recreation. Their first product, the Transition, is a light aircraft that can convert from a car to a plane in just a few minutes.'),
 'company_slogan': '  Sure, here\'s a slogan for Terrafugia:\n\n"Terrafugia: Take to the skies, and hit the road"\n\nOr, if you prefer, here\'s another option:\n\n"Terrafugia: The sky\'s the limit, and so is the highway"\n\nI hope these options capture the essence of Terrafugia\'s innovative technology and the freedom it offers to its customers!'}

#### Sequential Chain

![Sequential Chain](./images/SequentialChain.PNG)

In [4]:
code_generation_prompt = ChatPromptTemplate.from_messages(
   [
       SystemMessage(content="You are a code assistant that generates code for users input"),
       HumanMessagePromptTemplate.from_template("generate code for {text} in {language}")

   ] 
)

time_complexity_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a code assistant that evaluates the time and space complexity of code for users input"),
        HumanMessagePromptTemplate.from_template("{code}")
    ]
)


code_summary_chain = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="you are a code assitant that prints the time complexity of the code and code provided by the user and an explanation for the code"),
        HumanMessagePromptTemplate.from_template("{code} and {time_complexity} ")
    ]
)


In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = RunnableParallel(
    code_generation = code_generation_prompt | model,
    code_complexity = RunnablePassthrough.assign(code = (code_generation_prompt | model))| time_complexity_prompt | model,
    code_comparision = {'code' : (code_generation_prompt | model), 'time_complexity': ({'code' : code_generation_prompt | model} | time_complexity_prompt | model)} | code_summary_chain | model
)

In [11]:
chain.invoke({'text':'print prime numbers', 'language':'python'})

{'code_generation': AIMessage(content="  Sure! Here's an example code that prints all prime numbers up to a given number `n`:\n```\ndef print_primes(n):\n    for i in range(2, n+1):\n        is_prime = True\n        for j in range(2, int(i**0.5) + 1):\n            if i % j == 0:\n                is_prime = False\n                break\n        if is_prime:\n            print(i)\n```\nHere's an explanation of the code:\n\n1. The function `print_primes` takes an integer `n` as input.\n2. The loop `for i in range(2, n+1)` iterates over the numbers from 2 to `n`.\n3. The variable `is_prime` is initialized to `True` for each iteration.\n4. The loop `for j in range(2, int(i**0.5) + 1)` checks whether `i` is divisible by any number between 2 and the square root of `i`. If `i` is divisible by `j`, then `is_prime` is set to `False` and the loop breaks.\n5. If `is_prime` is still `True` after the inner loop, then `i` is a prime number and it is printed.\n\nHere's an example usage of the function

#### Router Chain

![Router Chain](./images/RouterChain.PNG)

In [22]:
sales_prompt = ChatPromptTemplate.from_messages(

    [

    SystemMessage(content="You are a helpfull, honest sales assisnt for a telecom company called brightspeed"),
    HumanMessagePromptTemplate.from_template("{text}")

    ]
    
)

service_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="You are a helpfull, honest service assisnt for a telecom company called brightspeed guide the user to resolve the issues."),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)



router_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="Categorize the user input to sales or service along with actual text"),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from transformers import pipeline


# classifier = pipeline('zero-shot-classification', candidate_labels = ['sales', 'service'])

parser = StrOutputParser()

chain = router_prompt | model | parser

def categorize_text(text):
    if("sales" in text):
        return sales_prompt
    else:
        return service_prompt
    

router_chain = RunnablePassthrough.assign(text = (router_prompt | model | parser)) | RunnableLambda(lambda text: categorize_text(text)) | model

In [24]:
router_chain.invoke({'text':"What are the broadband plans available?"})

AIMessage(content="  Hello! Welcome to Brightspeed! We're happy to help you with your inquiry.\n\nWe offer a variety of broadband plans that cater to different needs and budgets. Our plans come with different speeds, data caps, and features to ensure you get the best experience for your online activities.\n\nHere are some of our most popular broadband plans:\n\n1. Basic: This plan offers speeds of up to 50 Mbps and a data cap of 500 GB. It's perfect for light internet users who primarily use the internet for browsing, emailing, and social media.\n2. Standard: With speeds of up to 100 Mbps and a data cap of 1 TB, this plan is ideal for households with multiple devices. It's great for streaming, online gaming, and heavy internet usage.\n3. Premium: Our top-tier plan offers speeds of up to 500 Mbps and a data cap of 2 TB. It's designed for heavy internet users who need fast speeds for streaming, online gaming, and large file transfers.\n\nWe also offer customized plans for businesses, so 

#### Transform Chain

- used to transform the input before it is passed to the prompt or model.

In [29]:
def to_upper(input):
    return {'text':str(input).upper()}

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="you are a helpfull assistant"),
        HumanMessagePromptTemplate.from_template("{text}, print the actual text")
    ]
    
) 

chain = RunnableLambda(lambda text: to_upper(text)) | prompt | model

chain.invoke({"text":"Its me Hi!"})

AIMessage(content='  Sure, I\'d be happy to help! The actual text you provided is:\n\n"ITS ME HI!"\n\nIs there anything else I can assist you with?')

#### Math Chain

In [ ]:
! pip install numexpr

In [33]:
from langchain.chains import LLMMathChain

chain = LLMMathChain.from_llm(llm)

chain.invoke({"What is 2+2?"})

{'question': {'What is 2+2?'}, 'answer': 'Answer: 4'}

# LangChain Legacy Chains

In [ ]:
# Sequential Chains

from langchain.chains import SimpleSequentialChain, LLMChain

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
)

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Provide a slogan for the company {company_name}"
)

chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
)

In [ ]:
simple_seq_chain = SimpleSequentialChain(chains=[chain_one,chain_two], verbose=True)

simple_seq_chain.run("invisible car")

In [ ]:
from langchain.chains import SequentialChain

language_translation_prompt = ChatPromptTemplate.from_template(
    "translate the {review} to english"
)

review_prompt=ChatPromptTemplate.from_template(
    "provide a summary for the {english_review} in 20 words"
)

language_recognition_prompt = ChatPromptTemplate.from_template(
    "what language is the review {review} in?"
)

final_response_prompt = ChatPromptTemplate.from_template(
    """write a follow up response to the following
    summary in the specific langauge: {summary} \n langauge : {language}
    """
)

translation_chain = LLMChain(
    llm=llm,
    prompt=language_translation_prompt,
    output_key="english_review",
)
review_chain = LLMChain(
    llm=llm,
    prompt=review_prompt,
    output_key="summary",

)
language_recognition_chain = LLMChain(
    llm=llm,
    prompt=language_recognition_prompt,
    output_key="language",
)
final_response_chain= LLMChain(
    llm=llm,
    prompt=final_response_prompt,
    output_key="followup_message",
)

sequential_chain= SequentialChain(
    chains=[translation_chain,review_chain,language_recognition_chain,final_response_chain],
    input_variables=["review"],
    output_variables=["english_review","summary","followup_message","language"],
    verbose=True,
)

In [ ]:
sequential_chain("text")

In [ ]:
# Router chain

positive_review_prompt = "provide a response thanking the user for the {input}"

negative_review_prompt = "Provide a solution for the {input} and apologise to the customer"

prompt_info = [
    {
        "name":"thank you response",
        "description" :"A thank you note for positive review from the customer",
        "prompt_template":positive_review_prompt
    },
    {
        "name":"apology",
        "description" :"An apology message to the customer for a negative review or statemnt from the customer",
        "prompt_template":negative_review_prompt
      
    }
]

In [ ]:
from langchain.prompts import PromptTemplate

destination_chains = {}

for p in prompt_info:
    name = p['name']
    prompt = ChatPromptTemplate.from_template(template=p['prompt_template'])
    chain = LLMChain(prompt=prompt, llm=llm)
    destination_chains[name]=chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_info]
destination_str = "\n".join(destinations)

In [ ]:
dedault_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=dedault_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
from langchain.chains import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
router_prompt = PromptTemplate(template=router_template,input_variables=["input"], output_parser=RouterOutputParser())

In [ ]:
llm_router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [ ]:
final_router_chain = MultiPromptChain(router_chain=llm_router_chain,
                                      default_chain=default_chain,
                                      verbose=True,
                                      destination_chains=destination_chains)

final_router_chain.run("Do you sell potatos?")

task: replace strng output parser with JSON and Structured Output parser

In [ ]:
from langchain_core.pydantic_v1 import BaseModel,Field

class Code(BaseModel):
    text: str = Field(description="")
    # add necessary fields

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser(pydantic_object=Code)

In [ ]:
## Structured Output Parser


from langchain.output_parsers import StructuredOutputParser, ResponseSchema


response_schema = [ResponseSchema(
    # add necessary fields
)]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

output_parser.schema()

In [35]:
! pip install pypdf


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pypdf-4.0.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.0.1-py3-none-any.whl (283 kB)


In [4]:
# Pdf Loader

from langchain.document_loaders import PyPDFLoader


loader = PyPDFLoader('C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-5/assets/the-velveteen-rabbit.pdf')

pages = loader.load_and_split()

pages

[Document(page_content='The Velveteen RabbitByMargery Williams', metadata={'source': 'C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-5/assets/the-velveteen-rabbit.pdf', 'page': 0}),
 Document(page_content="There was once a velveteen rabbit, and in the beginning he was really splendid. He was fat and bunchy, as a rabbit should be; his coat was spotted brown and white, he had real thread whiskers, and his ears were lined with pink sateen. On Christmas morning, when he sat wedged in the top of the Boy's stocking, with a sprig of holly between his paws, the effect was charming.There were other things in the stocking, nuts and oranges and a toy engine, and chocolate almonds and a clockwork mouse, but the Rabbit was quite the best of all. For at least two hours the Boy loved him, and then Aunts and Uncles came to dinner, and there was a great rustling of tissue paper and unwrapping of parcels, and in the excitement of looking at all the new presents t

In [5]:
# text spliting using character splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    separator='\n'
)

documents = text_splitter.split_documents(pages)

documents

Created a chunk of size 2107, which is longer than the specified 1000
Created a chunk of size 1625, which is longer than the specified 1000
Created a chunk of size 1857, which is longer than the specified 1000
Created a chunk of size 1859, which is longer than the specified 1000
Created a chunk of size 1862, which is longer than the specified 1000
Created a chunk of size 1276, which is longer than the specified 1000
Created a chunk of size 1376, which is longer than the specified 1000
Created a chunk of size 1984, which is longer than the specified 1000
Created a chunk of size 1767, which is longer than the specified 1000
Created a chunk of size 1945, which is longer than the specified 1000
Created a chunk of size 1599, which is longer than the specified 1000
Created a chunk of size 1423, which is longer than the specified 1000


[Document(page_content='The Velveteen RabbitByMargery Williams', metadata={'source': 'C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-5/assets/the-velveteen-rabbit.pdf', 'page': 0}),
 Document(page_content="There was once a velveteen rabbit, and in the beginning he was really splendid. He was fat and bunchy, as a rabbit should be; his coat was spotted brown and white, he had real thread whiskers, and his ears were lined with pink sateen. On Christmas morning, when he sat wedged in the top of the Boy's stocking, with a sprig of holly between his paws, the effect was charming.There were other things in the stocking, nuts and oranges and a toy engine, and chocolate almonds and a clockwork mouse, but the Rabbit was quite the best of all. For at least two hours the Boy loved him, and then Aunts and Uncles came to dinner, and there was a great rustling of tissue paper and unwrapping of parcels, and in the excitement of looking at all the new presents t

In [41]:
# text spliting using  recursive character splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    separators=['\n', '\n\n']
)

documents = text_splitter.split_documents(pages)

documents

[Document(page_content='The Velveteen RabbitByMargery Williams', metadata={'source': 'C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-5/assets/the-velveteen-rabbit.pdf', 'page': 0}),
 Document(page_content="There was once a velveteen rabbit, and in the beginning he was really splendid. He was fat and bunchy, as a rabbit should be; his coat was spotted brown and white, he had real thread whiskers, and his ears were lined with pink sateen. On Christmas morning, when he sat wedged in the top of the Boy's stocking, with a sprig of holly between his paws, the effect was charming.There were other things in the stocking, nuts and oranges and a toy engine, and chocolate almonds and a clockwork mouse, but the Rabbit was quite the best of all. For at least two hours the Boy loved him, and then Aunts and Uncles came to dinner, and there was a great rustling of tissue paper and unwrapping of parcels, and in the excitement of looking at all the new presents t

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# embedding model from hf
embeddings = HuggingFaceEmbeddings(
    model_name='thenlper/gte-large'
)

index = 'trng-index'

result = [ Pinecone.from_documents(documents,embeddings.embed_documents, index_name =index)]

result

In [ ]:
import pinecone
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv('PINECONE_API_KEY')

client = Pinecone(api_key=key)

index = client.Index('trng-index')

In [59]:
from langchain.vectorstores.pinecone import Pinecone
from langchain.chains import RetrievalQA


vectordb = Pinecone(index, embeddings.embed_query, "text")

retreiver = vectordb.as_retriever()

chain = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retreiver)

chain.run("rabbit")

c:\TrainingMaterial\generative-ai\genai-material\.venv\Lib\site-packages\langchain_community\vectorstores\pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


'  Sure! Here\'s a summary of the story of "The Velveteen Rabbit" based on the provided text:\n\nThe story begins with a description of a velveteen rabbit, who is one of the toys in a boy\'s stocking on Christmas morning. The rabbit is initially loved and played with, but eventually gets forgotten as the boy receives other presents. The rabbit lives in the toy cupboard or on the nursery floor and is shy and awkward, feeling inferior to the other toys. One day, the rabbit is taken by the boy to a wood where he sees some real rabbits playing. The velveteen rabbit longs to join them but is too shy. He returns to the boy\'s home and is left alone in the nursery.\n\nThe story then jumps to the velveteen rabbit sitting in the bracken, watching two strange rabbits creep out of the tall bracken near him. They are rabbits like himself, but furry and brand-new. They twitch their noses and creep close to him, and the velveteen rabbit is curious but too shy to move. He eventually scratches his nos

In [ ]:
from langchain.retrievers import MultiQueryRetriever

retreiver = MultiQueryRetriever.from_llm(
    retriever= retreiver,
    llm = model
    )

retreiver.get_relevant_documents("the boy")